In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [2]:
dataset_path = "/kaggle/input/intel-image-dataset/Intel Image Dataset"
image_height = 150
image_width = 150
batch_size = 32

In [3]:
train_images = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    batch_size=batch_size,
    image_size=(image_height, image_width),
)

Found 3000 files belonging to 6 classes.
Using 2400 files for training.


In [4]:
val_images = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    batch_size=batch_size,
    image_size=(image_height, image_width),
)

Found 3000 files belonging to 6 classes.
Using 600 files for validation.


In [5]:
AUTOTUNE = tf.data.AUTOTUNE
train_images = train_images.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_images = val_images.cache().prefetch(buffer_size=AUTOTUNE)

In [6]:
def normalization(image, labels):
    image = tf.cast(image, tf.float32)/255.0
    return image, labels

def augment_data(image, labels):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    return image, labels

train_images = train_images.map(normalization).map(augment_data)
val_images = val_images.map(normalization)

In [7]:
class_names = ["buildings", "forest", "glacier", "mountain", "sea", "street"]

In [8]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 5, 5, 512)      │     1,180,160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,568,576 (5.98 MB)

 Trainable params: 1,568,576 (5.98 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.2)),
model.add(layers.Dense(10))

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 5, 5, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       819,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,388,490 (9.11 MB)

 Trainable params: 2,388,490 (9.11 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

model.compile(optimizer=Adam(learning_rate=lr_schedule), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [13]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(train_images, epochs=20, validation_data=(val_images), callbacks=[early_stopping])

Epoch 1/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.2333 - loss: 1.9980 - val_accuracy: 0.4117 - val_loss: 1.4382
Epoch 2/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.4015 - loss: 1.5246 - val_accuracy: 0.5000 - val_loss: 1.2630
Epoch 3/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.4542 - loss: 1.3959 - val_accuracy: 0.5517 - val_loss: 1.2322
Epoch 4/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.5033 - loss: 1.2787 - val_accuracy: 0.5900 - val_loss: 1.0636
Epoch 5/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.5301 - loss: 1.2360 - val_accuracy: 0.4917 - val_loss: 1.2298
Epoch 6/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.5551 - loss: 1.2039 - val_accuracy: 0.6550 - val_loss: 0.9724
Epoch 7/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.5921 - loss: 1.0754 - val_accuracy: 0.6033 - val_loss: 1.0151
Epoch 8/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.5938 - loss: 1.0742 - val_accuracy: 0.6583 - val_loss:

In [14]:
val_loss, val_accuracy = model.evaluate(val_images, verbose=2)
print(f'The accuracy is: {val_accuracy}')

19/19 - 5s - 275ms/step - accuracy: 0.7383 - loss: 0.6909
The accuracy is: 0.7383333444595337
